**ANN Project**

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Prepping the dataset for training the model

#### Cleaning milage, price, and dropping some columns

In [ ]:
# Load the dataset
used_cars = pd.read_csv('used_cars.csv')

used_cars = used_cars.drop(columns=['int_col', 'clean_title'])

print("Before dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars = used_cars.dropna()

print("After dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars['price'] = used_cars['price'].str.replace('$', '').str.replace(',', '').astype(int)
used_cars['milage'] = used_cars['milage'].str.replace('mi.', '').str.replace(',', '').astype(int)

used_cars.head(10)

Before dropping rows with NaN values:
4009
After dropping rows with NaN values:
3730
it worked


,brand,model,model_year,milage,fuel_type,engine,ext_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,Black,At least 1 accident or damage reported,10300
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,Moonlight Cloud,At least 1 accident or damage reported,38005
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Blue,None reported,54598
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,Black,None reported,15500
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,None reported,34999
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,Silver,None reported,14798
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Blue,None reported,31000
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,Green,None reported,7300
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,Black,None reported,41927
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Fuji White,None reported,73897


#### Cleaning the accident column

In [7]:
print(used_cars['accident'].unique())

def clean_accident(text):
    if pd.isna(text):
        return None
    elif "None reported" in text:
        return 0
    else:
        return 1
    
used_cars["accident_reported"] = used_cars["accident"].apply(clean_accident)
used_cars = used_cars.dropna(subset=["accident_reported"])
used_cars = used_cars.drop(columns=["accident"])

print(used_cars['accident_reported'].value_counts(dropna=False))

used_cars.head(10)


['At least 1 accident or damage reported' 'None reported']
accident_reported
0    2752
1     978
Name: count, dtype: int64


,brand,model,model_year,milage,fuel_type,engine,ext_col,price,accident_reported
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,Black,10300,1
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,Moonlight Cloud,38005,1
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Blue,54598,0
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,Black,15500,0
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,34999,0
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,Silver,14798,0
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Blue,31000,0
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,Green,7300,0
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,Black,41927,0
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Fuji White,73897,0


Cleaning Brand

In [ ]:
# Assign unique numbers to each brand
used_cars['brand_category'] = pd.Categorical(used_cars['brand']).codes + 1


columns_to_drop = [col for col in used_cars.columns if col.startswith('brand_') and col != 'brand_category']
used_cars = used_cars.drop(columns=columns_to_drop)

# Create a mapping table for brand names and their corresponding category numbers
brand_mapping = used_cars[['brand', 'brand_category']].drop_duplicates().sort_values(by='brand_category')

# Display the mapping table
print(brand_mapping)

used_cars.head(10)

              brand  brand_category
5             Acura               1
151            Alfa               2
11            Aston               3
4              Audi               4
7               BMW               5
40          Bentley               6
229         Bugatti               7
492           Buick               8
67         Cadillac               9
21        Chevrolet              10
55         Chrysler              11
17            Dodge              12
704            FIAT              13
152         Ferrari              14
0              Ford              15
101             GMC              16
82          Genesis              17
41            Honda              18
52           Hummer              19
1           Hyundai              20
3          INFINITI              21
14           Jaguar              22
35             Jeep              23
33              Kia              24
76      Lamborghini              25
10             Land              26
2             Lexus         

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price,brand_category
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,At least 1 accident or damage reported,10300,15
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,At least 1 accident or damage reported,38005,20
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,None reported,54598,27
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,None reported,15500,21
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,None reported,34999,4
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,F,Silver,None reported,14798,1
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,None reported,31000,4
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,None reported,7300,5
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,None reported,41927,27
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Automatic,Fuji White,None reported,73897,26
